# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [3]:
#cities_df.count()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

In [6]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
cities_weather = cities_df.dropna()
weather_criteria = cities_weather.loc[(cities_weather['Cloudiness'] > 0) & (cities_weather['Wind Speed'] < 10)] 
weather_criteria_df = cities_weather.loc[(cities_weather['Max Temp'] > 70) & (cities_weather['Max Temp'] <= 80)]
weather_criteria_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
28,28,rikitea,95,PF,1558378669,76,-23.12,-134.97,74.58,12.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df =  pd.DataFrame(weather_criteria_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
1,mahebourg,MU,-20.41,57.70,
12,avarua,CK,-21.21,-159.78,
18,bambous virieux,MU,-20.34,57.76,
25,kantang,TH,7.42,99.52,
28,rikitea,PF,-23.12,-134.97,


In [24]:
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [25]:
hotel_name = []

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params).json()
    results = response['results']

    try:
        hotel_name.append(response["results"][0]['name'])

    except IndexError:
        hotel_name.append(np.nan)

In [29]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
1,mahebourg,MU,-20.41,57.70,Holiday Inn Mauritius Mon Tresor
12,avarua,CK,-21.21,-159.78,The Edgewater Resort and Spa
18,bambous virieux,MU,-20.34,57.76,Laguna Beach Hotel & Spa
25,kantang,TH,7.42,99.52,NaN
28,rikitea,PF,-23.12,-134.97,Pension Maro'i


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in weather_criteria_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [31]:
# marker layer
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))